In [300]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqUtils import CodonUsage
from Bio.SeqUtils import IUPACData

import random
import csv
import numpy as np
from collections import Counter

from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

from scipy.special import softmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [274]:
random.seed(0)

In [240]:
bases = "ATCG"
codons = [a + b + c for a in bases for b in bases for c in bases]
codon_dict = {}
for codon in codons:
    codon_dict[codon] = 0.0

In [368]:
backward_codon_dict = {}

for codon in codons:
    AA = str(Seq(codon).translate())
    backward_codon_dict[AA] = []

for codon in codons:
    AA = str(Seq(codon).translate())
    backward_codon_dict[AA].append(codon)

In [278]:
ecoli_train = np.loadtxt("../data/data_split/ecoli_heg_train.txt", dtype="str")
ecoli_test = np.loadtxt("../data/data_split/ecoli_heg_test.txt", dtype="str")

In [593]:
def seq_to_triplet(seq):
    return [(seq[i:i+3]) for i in range(0, len(seq), 3)] 

def seqlist_to_triplets(seqs_NT):
    ''' returns list of list of codons given list of NT sequences

        Args:
            list(str): NT sequences

        Returns:
            list(list(str)): list of list of codons
    '''
    return [seq_to_triplet(seq) for seq in seqs_NT]

def NTlist_to_AA(seqs_NT):
    ''' returns list of AA sequences given list of NT sequences

        Args:
            list(str): NT sequences

        Returns:
            list(str): list of AA sequences
    '''
    AA_seqs = []
    for seq in seqs_NT:
        AA_seq = list(str(Seq(seq).translate()))
        #makes sure all start codon code for methionine (in case alternative start codons exist)
        AA_seq[0] = 'M'
        AA_seqs.append("".join(AA_seq))
  
    return AA_seqs

def pad_for_trigram(seqs):
    ''' returns padded list of list of sequences for trigrams

        Args:
            list(str): list of AA sequences

        Returns:
            list(list(char)): list of list of AA sequences, each padded left and right by 1 <s>, </s>
    '''
    padded_seqs = []
    for seq in seqs:
        padded_seq = list(pad_sequence(seq, pad_left=True, left_pad_symbol="<s>", 
                                    pad_right=True, right_pad_symbol="</s>", n=2))
        padded_seqs.append(padded_seq)
        
    return padded_seqs

def pad_for_fivegram(seqs):
    ''' returns padded list of list of sequences for fivegrams

        Args:
            list(str): list of AA sequences

        Returns:
            list(list(char)): list of list of AA sequences, each padded left and right by 2 <s>, </s>
    '''
    padded_seqs = []
    for seq in seqs:
        padded_seq = list(pad_sequence(seq, pad_left=True, left_pad_symbol="<s>", 
                                    pad_right=True, right_pad_symbol="</s>", n=3))
        padded_seqs.append(padded_seq)
        
    return padded_seqs

def unigram_dictionary(seq_list):
    ''' returns dictionary that maps unigram to most frequent codon

        Args:
            list(str): list of NT sequences

        Returns:
            dict{amino acid, dict{codon, frequency}}
    '''
    seqs_AA = NTlist_to_AA(seq_list)
    seqs_codon = seqlist_to_triplets(seq_list)
    
    unigram_dict = {}
    
    for i in range(len(seqs_AA)): #construct a dictionary that maps amino acid to list of matching codons
        seq_codon = seqs_codon[i]
        seq_AA_unigrams = list(ngrams(seqs_AA[i], n=1))
        for j in range(len(seq_AA_unigrams)):
            unigram = "".join(seq_AA_unigrams[j])
            if unigram in unigram_dict:
                unigram_dict[unigram].append(seq_codon[j])
            else:
                unigram_dict[unigram] = [seq_codon[j]]
    
    unigram_dict_frequency = {}
    
    for key in unigram_dict.keys(): #converts above dictionary item into frequency Counter object
        codon_counter = Counter(unigram_dict[key])
        total_count = 0
        for codon_key in codon_counter:
            total_count += codon_counter[codon_key]
        
        for codon_key in codon_counter:
            codon_counter[codon_key] = codon_counter[codon_key] / total_count

        unigram_dict_frequency[key] = codon_counter
    
    return unigram_dict_frequency

def trigram_dictionary(seq_list):
    ''' returns dictionary that maps trigram to most frequent codon

        Args:
            list(str): list of NT sequences

        Returns:
            dict{AA Trigram, dict{codon, frequency}}
    '''
    seqs_AA = NTlist_to_AA(seq_list)
    seqs_codon = seqlist_to_triplets(seq_list)
    seqs_AA_padded = pad_for_trigram(seqs_AA)
    
    trigram_dict = {}
    
    for i in range(len(seqs_AA_padded)): # construct a dictionary that maps amino acid trigram to list of matching codons
        seq_codon = seqs_codon[i]
        seq_AA_trigrams = list(ngrams(seqs_AA_padded[i], n=3))
        for j in range(len(seq_AA_trigrams)):
            trigram = "".join(seq_AA_trigrams[j])
            if trigram in trigram_dict:
                trigram_dict[trigram].append(seq_codon[j])
            else:
                trigram_dict[trigram] = [seq_codon[j]]
    
    trigram_dict_frequency = {}
    
    for key in trigram_dict.keys():
        codon_counter = Counter(trigram_dict[key])
        total_count = 0
        for codon_key in codon_counter:
            total_count += codon_counter[codon_key]
        
        for codon_key in codon_counter:
            codon_counter[codon_key] = codon_counter[codon_key] / total_count

        trigram_dict_frequency[key] = codon_counter
    
    return trigram_dict_frequency


def fivegram_dictionary(seq_list):
    ''' returns dictionary that maps fivegram to most frequent codon

        Args:
            list(str): list of NT sequences

        Returns:
            dict{AA Fivegram, dict{codon, frequency}}
    '''
    seqs_AA = NTlist_to_AA(seq_list)
    seqs_codon = seqlist_to_triplets(seq_list)
    seqs_AA_padded = pad_for_fivegram(seqs_AA)
    
    fivegram_dict = {}
    
    for i in range(len(seqs_AA_padded)): #construct a dictionary that maps amino acid fivegram to list of matching codons
        seq_codon = seqs_codon[i]
        seq_AA_fivegrams = list(ngrams(seqs_AA_padded[i], n=5))
        for j in range(len(seq_AA_fivegrams)):
            fivegram = "".join(seq_AA_fivegrams[j])
            if fivegram in fivegram_dict:
                fivegram_dict[fivegram].append(seq_codon[j])
            else:
                fivegram_dict[fivegram] = [seq_codon[j]]
    
    fivegram_dict_frequency = {}
    
    for key in fivegram_dict.keys():
        codon_counter = Counter(fivegram_dict[key])
        total_count = 0
        for codon_key in codon_counter:
            total_count += codon_counter[codon_key]
        
        for codon_key in codon_counter:
            codon_counter[codon_key] = codon_counter[codon_key] / total_count

        fivegram_dict_frequency[key] = codon_counter
    
    return fivegram_dict_frequency

In [280]:
# split train into train/val to figure out the weights
ecoli_train_train, ecoli_train_test = train_test_split(ecoli_train, test_size=0.3)

In [465]:
# train frequency tables solely from the train set of train set
unigram_frequency_dict = unigram_dictionary(ecoli_train_train)
trigram_frequency_dict = trigram_dictionary(ecoli_train_train)
fivegram_frequency_dict = fivegram_dictionary(ecoli_train_train)

In [515]:
def predict_codons(AA_seq, codon_dict, unigram_frequency_dict, trigram_frequency_dict, fivegram_frequency_dict, a=1/3, b=1/3, c=1/3):
    ''' returns list of predicted codon probabilities given amino acid sequence

        Args:
            list(str): list of NT sequences
            dict{codon, frequency}
            unigram_frequency_dict (as above)
            trigram_frequency_dict (as above)
            fivegram_frequency_dict (as above)
            unigram weight
            trigram weight
            fivegram weight
            

        Returns:
            list{list(flaot)}: list of list codon probabilities
    '''
    trigram_padded = list(pad_sequence(AA_seq, pad_left=True, left_pad_symbol="<s>", 
                                    pad_right=True, right_pad_symbol="</s>", n=2))
    
    fivegram_padded = list(pad_sequence(AA_seq, pad_left=True, left_pad_symbol="<s>", 
                                    pad_right=True, right_pad_symbol="</s>", n=3))
    
    unigrams = ["".join(ngram) for ngram in list(ngrams(AA_seq, n=1))]
    trigrams = ["".join(ngram) for ngram in list(ngrams(trigram_padded, n=3))]
    fivegrams = ["".join(ngram) for ngram in list(ngrams(fivegram_padded, n=5))]
    
    prediction_list = []
    
    for i in range(len(unigrams)): #loop over each AA in the sequence, generate a dictionary for each AA
        
        unigram = unigrams[i] #single amino acid at this position
        trigram = trigrams[i] #3 amino acids concatenated at this position
        fivegram = fivegrams[i] # 5 amino acids concatenated at this position
        prediction = codon_dict.copy() #dict that maps possible codon (not all 20) to frequency
        
        for codon in codon_dict.keys(): #loop over possible codons given AA
            unigram_freq = 0.0
            trigram_freq = 0.0
            fivegram_freq = 0.0
            
            if unigram in unigram_frequency_dict and codon in unigram_frequency_dict[unigram]:
                unigram_freq = unigram_frequency_dict[unigram][codon]
                
            if trigram in trigram_frequency_dict and codon in trigram_frequency_dict[trigram]:
                trigram_freq = trigram_frequency_dict[trigram][codon]
            
            if fivegram in fivegram_frequency_dict and codon in fivegram_frequency_dict[fivegram]:
                fivegram_freq = fivegram_frequency_dict[fivegram][codon]
                
            prediction[codon] = a * unigram_freq + b * trigram_freq + c * fivegram_freq
        
        #normalize
        prediction_probs = np.array(list(prediction.values())) * 1.0 / sum(list(prediction.values()))
            
        prediction_list.append(prediction_probs)
    
    return prediction_list

In [597]:
def calculate_loss(NT_seqs, codon_dict, unigram_frequency_dict, trigram_frequency_dict, fivegram_frequency_dict, a=1/3, b=1/3, c=1/3):
    AA_seqs = NTlist_to_AA(NT_seqs)
    prediction_list = []
    correct_list = []
    loss = 0.0
    
    for AA_seq in AA_seqs:
        for prediction in predict_codons(AA_seq, codon_dict, unigram_frequency_dict, trigram_frequency_dict, fivegram_frequency_dict, a, b, c):
            prediction_list.append(prediction)
            
    for NT_seq in NT_seqs:
        codon_seq = seq_to_triplet(NT_seq)
        for codon in codon_seq:
            target_codon_dict = codon_dict.copy()
                
            target_codon_dict[codon] = 1.0
            correct_list.append(list(target_codon_dict.values()))
        
    
    for i in range(len(correct_list)):
 
        loss += log_loss(correct_list[i], prediction_list[i])
    
    return loss

In [468]:
min_loss = 10000000
curr_w1 = 0
curr_w2 = 0
curr_w3 = 0

keep_track = []

for w1 in [0.0, 0.1, 0.2, 0.3, 1.0/3.0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for w2 in [0.0, 0.1, 0.2, 0.3, 1.0/3.0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        if (w2 + w1 > 1.0):
            continue
        loss = calculate_loss(ecoli_train_test, codon_dict, unigram_frequency_dict, trigram_frequency_dict, fivegram_frequency_dict, a=w1, b=w2, c=1.0-w1-w2)
        keep_track.append(([w1, w2, 1.0 - w1-w2], loss))
        
        print ([w1, w2, 1.0 - w1-w2], loss)
        
        if loss < min_loss:
            min_loss = loss
            curr_w1 = w1
            curr_w2 = w2
            curr_w3 = 1.0-w1-w2

[0.1, 0.1, 0.8] 820.9434180197115
[0.1, 0.2, 0.7] 716.0510192756986
[0.1, 0.3, 0.6000000000000001] 648.3354819633129
[0.1, 0.3333333333333333, 0.5666666666666667] 630.7714007727541
[0.1, 0.4, 0.5] 601.124808565652
[0.1, 0.5, 0.4] 567.4824529398527
[0.1, 0.6, 0.30000000000000004] 544.0677776008355
[0.1, 0.7, 0.20000000000000007] 529.4255919930613
[0.1, 0.8, 0.09999999999999998] 523.8694131232288
[0.1, 0.9, 0.0] 534.5584832571271
[0.2, 0.1, 0.7000000000000001] 702.5130817199567
[0.2, 0.2, 0.6000000000000001] 631.274936023313
[0.2, 0.3, 0.5] 582.3296315139254
[0.2, 0.3333333333333333, 0.46666666666666673] 569.4060218475373
[0.2, 0.4, 0.4] 547.5575062272468
[0.2, 0.5, 0.30000000000000004] 523.1850468738621
[0.2, 0.6, 0.20000000000000007] 507.46768181647013
[0.2, 0.7, 0.10000000000000009] 500.212841762099
[0.2, 0.8, 0.0] 504.8673200056706
[0.3, 0.1, 0.6] 626.7798543697135
[0.3, 0.2, 0.49999999999999994] 574.8890511752146
[0.3, 0.3, 0.39999999999999997] 538.4171592354461
[0.3, 0.333333333333

In [612]:
def get_accuracy_unigram(ecoli_train, ecoli_test, backward_codon_dict):
    unigram_frequency_dict = unigram_dictionary(ecoli_train)
    
    AA_seqs = NTlist_to_AA(ecoli_test)
    codon_seqs = seqlist_to_triplets(ecoli_test)
    
    correct = 0
    total = 0
    
    output = []
    
    for i in range(len(AA_seqs)):
        predicted_seq = []
        
        AA_seq_unigrams = list(ngrams(AA_seqs[i], n=1))
        for j in range(len(AA_seq_unigrams)):
            ngram_concat = "".join(AA_seq_unigrams[j])
            total += 1
            if ngram_concat in unigram_frequency_dict:
                pred = unigram_frequency_dict[ngram_concat].most_common()[0][0]
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
            else:
                #predict randomly
                pred = random.choice(backward_codon_dict[AA_seqs[i][j]])
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
        
        output.append("".join(predicted_seq))
    
    return correct/total, output

In [613]:
def get_accuracy_trigram(ecoli_train, ecoli_test, backward_codon_dict):
    trigram_frequency_dict = trigram_dictionary(ecoli_train)
    
    AA_seqs = NTlist_to_AA(ecoli_test)
    AA_seqs_padded = pad_for_trigram(AA_seqs)
    codon_seqs = seqlist_to_triplets(ecoli_test)
    
    correct = 0
    total = 0
    
    output = []
    
    for i in range(len(AA_seqs_padded)):
        predicted_seq = []
        
        AA_seq_trigrams = list(ngrams(AA_seqs_padded[i], n=3))
        for j in range(len(AA_seq_trigrams)):
            ngram_concat = "".join(AA_seq_trigrams[j])
            total += 1
            if ngram_concat in trigram_frequency_dict:
                pred = trigram_frequency_dict[ngram_concat].most_common()[0][0]
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
            else:
                #predict randomly
                pred = random.choice(backward_codon_dict[AA_seqs[i][j]])
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
        
        output.append("".join(predicted_seq))
    
    return correct/total, output

In [614]:
def get_accuracy_fivegram(ecoli_train, ecoli_test, backward_codon_dict):
    fivegram_frequency_dict = fivegram_dictionary(ecoli_train)
    
    AA_seqs = NTlist_to_AA(ecoli_test)
    AA_seqs_padded = pad_for_fivegram(AA_seqs)
    codon_seqs = seqlist_to_triplets(ecoli_test)
    
    correct = 0
    total = 0
    
    output = []
    
    for i in range(len(AA_seqs_padded)):
        predicted_seq = []
        
        AA_seq_fivegrams = list(ngrams(AA_seqs_padded[i], n=5))
        for j in range(len(AA_seq_fivegrams)):
            ngram_concat = "".join(AA_seq_fivegrams[j])
            total += 1
            if ngram_concat in fivegram_frequency_dict:
                pred = fivegram_frequency_dict[ngram_concat].most_common()[0][0]
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
            else:
                #predict randomly
                pred = random.choice(backward_codon_dict[AA_seqs[i][j]])
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
        output.append("".join(predicted_seq))
    
    return correct/total, output

In [615]:
def get_accuracy_fivegram(ecoli_train, ecoli_test, backward_codon_dict):
    fivegram_frequency_dict = fivegram_dictionary(ecoli_train)
    
    AA_seqs = NTlist_to_AA(ecoli_test)
    AA_seqs_padded = pad_for_fivegram(AA_seqs)
    codon_seqs = seqlist_to_triplets(ecoli_test)
    
    correct = 0
    total = 0
    
    output = []
    
    for i in range(len(AA_seqs_padded)):
        predicted_seq = []
        AA_seq_fivegrams = list(ngrams(AA_seqs_padded[i], n=5))
        
        for j in range(len(AA_seq_fivegrams)):
            ngram_concat = "".join(AA_seq_fivegrams[j])
            total += 1
            if ngram_concat in fivegram_frequency_dict:
                pred = fivegram_frequency_dict[ngram_concat].most_common()[0][0]
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
            else:
                #predict randomly
                pred = random.choice(backward_codon_dict[AA_seqs[i][j]])
                predicted_seq.append(pred)
                if codon_seqs[i][j] == pred:
                    correct += 1
        
        output.append("".join(predicted_seq))
    
    return correct/total, output

In [616]:
def get_accuracy_ngram(ecoli_train, ecoli_test, codon_dict, backward_codon_dict, w1, w2, w3):
    unigram_frequency_dict = unigram_dictionary(ecoli_train)
    trigram_frequency_dict = trigram_dictionary(ecoli_train)
    fivegram_frequency_dict = fivegram_dictionary(ecoli_train)
    
    AA_seqs = NTlist_to_AA(ecoli_test)
    codon_seqs = seqlist_to_triplets(ecoli_test)
    
    
    correct = 0
    total = 0
    
    output = []
    
    for i in range(len(AA_seqs)):
        predicted_seq = []
        
        AA_seq = AA_seqs[i]
        
        trigram_padded = list(pad_sequence(AA_seq, pad_left=True, left_pad_symbol="<s>", 
                                    pad_right=True, right_pad_symbol="</s>", n=2))
    
        fivegram_padded = list(pad_sequence(AA_seq, pad_left=True, left_pad_symbol="<s>", 
                                        pad_right=True, right_pad_symbol="</s>", n=3))

        unigrams = ["".join(ngram) for ngram in list(ngrams(AA_seq, n=1))]
        trigrams = ["".join(ngram) for ngram in list(ngrams(trigram_padded, n=3))]
        fivegrams = ["".join(ngram) for ngram in list(ngrams(fivegram_padded, n=5))]
        
        for j in range(len(AA_seq)):
            total += 1
            
            unigram = unigrams[j] #single amino acid at this position
            trigram = trigrams[j] #3 amino acids concatenated at this position
            fivegram = fivegrams[j] # 5 amino acids concatenated at this position
    
            prediction = codon_dict.copy()
            
            for codon in prediction.keys(): #loop over possible codons given AA
                unigram_freq = 0.0
                trigram_freq = 0.0
                fivegram_freq = 0.0

                if unigram in unigram_frequency_dict and codon in unigram_frequency_dict[unigram]:
                    unigram_freq = unigram_frequency_dict[unigram][codon]

                if trigram in trigram_frequency_dict and codon in trigram_frequency_dict[trigram]:
                    trigram_freq = trigram_frequency_dict[trigram][codon]

                if fivegram in fivegram_frequency_dict and codon in fivegram_frequency_dict[fivegram]:
                    fivegram_freq = fivegram_frequency_dict[fivegram][codon]

                prediction[codon] = w1 * unigram_freq + w2 * trigram_freq + w3 * fivegram_freq
            
            if (sum(list(prediction.values())) == 0.0):
                random_predict = random.choice(backward_codon_dict[AA_seq[j]])
                predicted_seq.append(random_predict)
                if (random_predict == codon_seqs[i][j]):
                    correct += 1
            
            elif (max(prediction, key=prediction.get) == codon_seqs[i][j]):
                predicted_seq.append(max(prediction, key=prediction.get))
                correct += 1
        output.append("".join(predicted_seq))
    
    return correct/total, output

In [619]:
print ("Unigram Train accuracy: ", get_accuracy_unigram(ecoli_train, ecoli_train, backward_codon_dict)[0])
print ("Unigram Test accuracy: ", get_accuracy_unigram(ecoli_train, ecoli_test, backward_codon_dict)[0])

Unigram Train accuracy:  0.6273062199579772
Unigram Test accuracy:  0.6253293896208658


In [622]:
print ("Trigram Train accuracy: ", get_accuracy_trigram(ecoli_train, ecoli_train, backward_codon_dict)[0])
print ("Trigram Test accuracy: ", get_accuracy_trigram(ecoli_train, ecoli_test, backward_codon_dict)[0])

Trigram Train accuracy:  0.6905707624557464
Trigram Test accuracy:  0.6318902930895401


In [623]:
print ("Fivegram Train accuracy: ", get_accuracy_fivegram(ecoli_train, ecoli_train, backward_codon_dict)[0])
print ("Fivegram Test accuracy: ", get_accuracy_fivegram(ecoli_train, ecoli_test, backward_codon_dict)[0])

Fivegram Train accuracy:  0.9832196413666062
Fivegram Test accuracy:  0.37601505781123956


In [626]:
print ("ngram Train accuracy: ", get_accuracy_ngram(ecoli_train, ecoli_train, codon_dict, backward_codon_dict, 0.7, 0.3, 0.0)[0])
print ("ngram Test accuracy: ", get_accuracy_ngram(ecoli_train, ecoli_test, codon_dict, backward_codon_dict, 0.7, 0.3, 0.0)[0])

ngram Train accuracy:  0.6652850934001093
ngram Test accuracy:  0.6410863135251411


In [627]:
unigram_output = get_accuracy_unigram(ecoli_train, ecoli_test, backward_codon_dict)[1]
trigram_output = get_accuracy_trigram(ecoli_train, ecoli_test, backward_codon_dict)[1]
fivegram_output = get_accuracy_fivegram(ecoli_train, ecoli_test, backward_codon_dict)[1]
ngram_output = get_accuracy_ngram(ecoli_train, ecoli_test, codon_dict, backward_codon_dict, 0.7, 0.3, 0.0)[1]

In [630]:
with open("ecoli_heg_unigram1.txt", "w") as txt_file:
    for line in unigram_output:
        txt_file.write(line + "\n")